In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [2]:
import sqlite3

## The Data

Let's start by having a look at our data. For now we will only display the first 10 rows of our table facts.

In [3]:
%%sql
SELECT * FROM facts LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46
6,ac,Antigua and Barbuda,442,442,0,92436,1.24,15.85,5.69,2.21
7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.0
8,am,Armenia,29743,28203,1540,3056382,0.15,13.61,9.34,5.8
9,as,Australia,7741220,7682300,58920,22751014,1.07,12.15,7.14,5.65
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56


This data is from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information. Below we describe a few column names of the above table to give you a feel of what kind of data the Factbook stores.

| Column                | Description  | 
|---|---|
| **name**              |  The name of the country |   
|**area**               | The total land and sea area of the country.|
|**population**         | The country's population.|
|**population_growth**  | The country's population growth as a percentage.|
|**birth_rate**         | The country's birth rate, or the number of births a year per 1,000 people.|
|**death_rate**         | The country's death rate, or the number of death a year per 1,000 people.|
|**area**               | The country's total area (both land and water).|
|**area_land**        | The country's land area in square kilometers.|
|**area_water**         | The country's waterarea in square kilometers.|

Next, we will calculate some summary statistics.

## Evaluating Some Summary Statistics

In [4]:
%%sql
SELECT MIN(population) min_pop, MAX(population) max_pop, MIN(population_growth) min_pop_growth,
       MAX(population_growth) max_pop_growth 
  FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


From the results above, we notice that there's a country with a population of 0 - which seems too odd.

Also, there is a country with a population almost the same as the world's population.

These two countries (rows) need to be investigated.

## Investigating the Suspicious Countries (rows): Outliers

In [5]:
%%sql
SELECT * FROM facts
 WHERE population == (SELECT MIN(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


And our output above clears the air. It makes sense that the population-of-zero row matches Antarctica. [The CIA Factbook for Antarctica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html) also confirms that to be true: there are no people living on the continent of Antarctica just of yet.

Now let's check the other odd row.

In [6]:
%%sql
SELECT * FROM facts
 WHERE population == (SELECT MAX(population) FROM facts );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


And, it turns out, as we suspected the row- with a population column of 7.3 billion- corresponds to the world's population.

## Summary Statistics Excluding the Suspicious Rows

In [7]:
%%sql
SELECT MIN(population) min_pop, MAX(population) max_pop, MIN(population_growth) min_pop_growth,
       MAX(population_growth) max_pop_growth 
  FROM facts
 WHERE name  NOT IN ('World', 'Antarctica') ;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
48,1367485388,0.0,4.02


Let's find the name of the country where the population is almost 1.4 billion.

In [8]:
%%sql
SELECT MAX(population) FROM facts
 WHERE name  NOT IN ('World', 'Antarctica') ;

 * sqlite:///factbook.db
Done.


MAX(population)
1367485388


In [9]:
%%sql
SELECT name FROM facts
 WHERE population == (SELECT MAX(population) FROM facts WHERE name  NOT IN ('World', 'Antarctica'));

 * sqlite:///factbook.db
Done.


name
China


There we have it. It's China!

## Inspecting Average Population and Area

When population and area are mention in one sentence- one aspect that comes to mind is population density. Here, we'll start to explore population density. 

First, we'll look at the averages for those respective columns: population and area.

Just as a reminder: we will need to exclude the rows for the whole world and Antarctica.

In [10]:
%%sql
SELECT AVG(population) avg_population, AVG(area) avg_area 
     FROM facts
 WHERE name NOT IN ('World', 'Antarctica');

 * sqlite:///factbook.db
Done.


avg_population,avg_area
32377011.0125,555093.546184739


We can see from the table of results above that on average a country hosts about 32 million humans.

Next, we dive deep into investigating the countries with distinguished population densities.

## Densely Populated Nations

We use the query below to identify those countries with average population and average area below (as well as above) the average figures we computed in the previous section.

In simple terms we are looking for those countries with relatively small areas and relatively very high population.

In [11]:
%%sql
SELECT * FROM facts
 WHERE population > (SELECT AVG(population) FROM facts )
   AND area < (SELECT AVG(area) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


Roughly looking through the table we find out that some of the countries are actually famous for their high population densities.

However, to have an accurate result we need to compute the population density index and base our selection on that.

Below are some of the questions I still need to explore; 

- What country has the most people? What country has the highest growth rate?
- Which countries have the highest ratios of water to land? Which countries have more water than land?
- Which countries will add the most people to their population next year?
- Which countries have a higher death rate than birth rate?
- What countries have the highest population/area ratio and how does it compare to list we found above?
